# Molecule Type ANOVA Analysis

This notebook performs ANOVA (Analysis of Variance) tests to compare binding-related features across different molecule types (Peptide, Nanobody, ScFV, Unidentified) from the EGFR competition data.

## Reproducibility Note
This notebook sets all relevant random seeds to ensure reproducible results.

In [1]:
import numpy as np
import pandas as pd
import random
from scipy import stats

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

print(f"Random seed set to: {RANDOM_SEED}")

Random seed set to: 42


In [2]:
import sys
import os

# Add the repo's scripts directory to Python path for theme import
theme_path = os.path.abspath('../adaptyv/adaptyv_analyses/scripts/plotting_python')
if theme_path not in sys.path:
    sys.path.insert(0, theme_path)

# Import the Adaptyv theme from the repo
import blog_post_theme as adaptyv_theme

# Apply the theme
adaptyv_theme.set_adaptyv_matplotlib_theme()

# Get color palettes
palettes = adaptyv_theme.get_adaptyv_palettes()

print(f"Theme loaded from repo: {theme_path}")

Theme loaded from repo: /Users/qamar/Downloads/adaptyv_nature_paper/egfr2024_post_competition/adaptyv/adaptyv_analyses/scripts/plotting_python


## Load and Prepare Data

In [3]:
import json

# Load the data from parquet file
df = pd.read_parquet("data/unioned_egfr_features.parquet")

print(f"Loaded {len(df)} rows with {len(df.columns)} columns")

Loaded 733 rows with 37 columns


In [4]:
def expand_feature_column(df: pd.DataFrame, feature_col: str = "feature") -> pd.DataFrame:
    """
    Expands a column containing dictionaries into separate columns.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    feature_col (str): The name of the column containing dictionaries.
    
    Returns:
    pd.DataFrame: A new DataFrame with the expanded features as separate columns.
    """
    df = df.copy()
    df[feature_col] = df[feature_col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Normalize and expand the feature column
    features_df = df[feature_col].apply(pd.Series)
    
    # Concatenate with the original DataFrame
    df = pd.concat([df.drop(columns=[feature_col]), features_df], axis=1)
    
    return df

In [5]:
# Expand alphapulldown features
df_pulldown = expand_feature_column(df, feature_col='alphapulldown_feature')

print(f"Expanded to {len(df_pulldown.columns)} columns")

Expanded to 54 columns


## Prepare Data for Statistical Analysis

In [6]:
# Filter out pacesa source and create categorical columns
df = df_pulldown[df_pulldown["source"] != 'pacesa']

# Create Source labels
df.loc[(df["source"].str.startswith("adaptyv")), "Source"] = "EGFR Competition"
df.loc[df["source"] == "biolm", "Source"] = "BioLM R1"
df.loc[df["source"] == "silica", "Source"] = "Bio x ML Team Silica"

# Create Data Source labels
df.loc[(df["source"] == "adaptyv-round-1"), "Data Source"] = "EGFR Competition R1"
df.loc[(df["source"] == "adaptyv-round-2"), "Data Source"] = "EGFR Competition R2"
df.loc[df["source"] == "biolm", "Data Source"] = "BioLM R1"
df.loc[df["source"] == "silica", "Data Source"] = "Bio x ML Team Silica"

# Create Control and Binding labels
df.loc[df["is_control"] == True, "Control"] = "Control"
df.loc[df["is_control"] != True, "Control"] = "Not Control"
df.loc[df["binding"] == True, "Binding"] = "True"
df.loc[df["binding"] == False, "Binding"] = "False"

ref_seqs = df[df["is_control"] == True].sequence.unique().tolist()

In [7]:
# Create Design Method categories
df.loc[(df.model_category.str.contains("proteinmpnn", na=False)), "Design Method"] = "MPNN"
df.loc[(df.model_category.str.contains("Evodiff", na=False)), "Design Method"] = "EvoDiff"
df.loc[df.model_category.str.contains("Md + docking", na=False), "Design Method"] = "Rosetta/Rational/Physics Informed"
df.loc[df.model_category.str.contains("Rational design", na=False), "Design Method"] = "Rosetta/Rational/Physics Informed"
df.loc[df.model_category.str.contains("Rosetta", na=False), "Design Method"] = "Rosetta/Rational/Physics Informed"
df.loc[df.model_category.str.contains("Rfdiffusion", na=False), "Design Method"] = "RFDiffusion"
df.loc[df.model_category.str.contains("Alphafold2", na=False), "Design Method"] = "Alphafold2/Bindcraft/RSO"
df.loc[df.model_category.str.contains("Bindcraft/rso", na=False), "Design Method"] = "Alphafold2/Bindcraft/RSO"
df.loc[df.model_category.str.contains("Esm2", na=False), "Design Method"] = "ESM2"
df.loc[df.model_category.str.contains("esm2", na=False), "Design Method"] = "ESM2"
df.loc[df.model_category.str.contains("Timed", na=False), "Design Method"] = "Timed"
df.loc[df.model_category.str.contains("mlde", na=False), "Design Method"] = "MLDE"
df.loc[df.model_category.str.contains("evoprotgrad", na=False), "Design Method"] = "EvoProtGrad"

## Molecule Type Classification

In [8]:
# Create Molecule Type labels
df["Molecule Type"] = df["molecule_type"]
df.loc[df["Molecule Type"] == "Polypeptide", "Molecule Type"] = "Unidentified"

print("Molecule Type counts:")
print(df["Molecule Type"].value_counts())

Molecule Type counts:
Molecule Type
Unidentified    234
Peptide         221
Nanobody        212
ScFV             66
Name: count, dtype: int64


In [9]:
# Create log-transformed binding affinity columns
df["log_kd"] = np.log10(df["kd_nm"])
df["log_kd_no_weak"] = df["log_kd"]
df.loc[df["log_kd"] == 4, "log_kd_no_weak"] = np.nan

## Prepare Data for ANOVA

In [10]:
# Create a copy for statistical analysis
df_stats = df.copy()

# Create binary Expression labels
df_stats.loc[df_stats.expression == "none", "Expression"] = "False"
df_stats.loc[df_stats.expression != "none", "Expression"] = "True"

# Rename columns for consistency
df_stats = df_stats.rename(columns={"expression": "Expression Strength", "binding_strength": "Binding Strength"})

## ANOVA Analysis Function

In [11]:
def run_anova_analysis(df, pairing_list, cat_targets, features):
    """
    Performs ANOVA tests for each categorical target within each molecule type pairing.
    
    Returns:
      A DataFrame with ANOVA results (F-statistic and p-value) for each feature.
    """
    result_dict = {}
    
    for pairing in pairing_list:
        pairing_label = '&'.join(map(str, pairing))
        if sorted(['Peptide', 'Unidentified', "Nanobody", "ScFV"]) == sorted(pairing):
            pairing_label = 'All'
        elif sorted(["Nanobody", "ScFV"]) == sorted(pairing):
            pairing_label = 'Immune'
        elif sorted(['Peptide', 'Unidentified']) == sorted(pairing):
            pairing_label = 'Not Immune'
        
        df_sub = df[df['Molecule Type'].isin(pairing)]
        
        for target in cat_targets:
            results = {}
            df_sub_target = df_sub.dropna(subset=[target])
            groups = df_sub_target.groupby(target)
            
            for feat in features:
                group_values = [group[feat].dropna().values for _, group in groups]
                
                if len(group_values) > 1 and all(len(vals) > 1 for vals in group_values):
                    try:
                        # ANOVA Test
                        F_stat, p_val = stats.f_oneway(*group_values)
                        anova_result = f"F={F_stat:.3f}, p={p_val:.3g}"
                        results[feat] = {"ANOVA": anova_result}
                    except Exception:
                        results[feat] = {"ANOVA": np.nan}
                else:
                    results[feat] = {"ANOVA": np.nan}
            
            result_dict[(pairing_label, target)] = pd.DataFrame(results).T
    
    results_df = pd.concat(result_dict, axis=1)
    
    return results_df

## Define Features and Pairings for Analysis

In [12]:
# Define categorical targets
cat_targets_binding = ["Binding"]

# Define features to analyze for binding
features_binding = [
    'esmc_600m_log_prob_normalized',
    'esm2_650m_log_prob_normalized',
    'iptm_ptm', 'iptm', 'pDockQ_mpDockQ',
    'average_interface_pae', 'average_interface_plddt', 'binding_energy',
    'Num_intf_residues', 'Polar', 'Hydrophobic', 'Charged',
    'contact_pairs', ' sc', ' hb', ' sb', ' int_solv_en', ' int_area',
    'pi_score'
]

# Define molecule type pairings
pairing_list = [
    ['Peptide', 'Unidentified', "Nanobody", "ScFV"],  # All
    ["Nanobody", "ScFV"],  # Immune (Ab-Domain)
    ['Peptide'],  # Peptide only
]

## Run ANOVA Analysis

In [13]:
# Run ANOVA on all sequences (including BioLM and Silica)
results_df_binding_wcheck = run_anova_analysis(
    df_stats, pairing_list, cat_targets_binding, features_binding
)

# Run ANOVA on competition sequences only
results_df_binding_comp_wcheck = run_anova_analysis(
    df_stats[df_stats.Source == "EGFR Competition"], 
    pairing_list, cat_targets_binding, features_binding
)

In [14]:
# Clean up column names
results_df_binding_comp_wcheck.columns = results_df_binding_comp_wcheck.columns.get_level_values(0)
results_df_binding_wcheck.columns = results_df_binding_wcheck.columns.get_level_values(0)

In [15]:
# Rename columns for clarity
results_df_binding_comp_wcheck = results_df_binding_comp_wcheck.rename(
    columns={"All": "All + Unidentified", "Immune": "Ab-Domain"}
)

# Add the ANOVA for Ab-Domains with BioLM and Silica data included
results_df_binding_comp_wcheck["Ab-Domain Extended"] = results_df_binding_wcheck["Immune"]

## Display and Save Results

In [16]:
# Display the final ANOVA table
results_df_binding_comp_wcheck

,All + Unidentified,Ab-Domain,Peptide,Ab-Domain Extended
esmc_600m_log_prob_normalized,"F=18.050, p=2.55e-05","F=0.052, p=0.82","F=8.760, p=0.00342","F=0.246, p=0.62"
esm2_650m_log_prob_normalized,"F=32.038, p=2.5e-08","F=0.085, p=0.771","F=29.297, p=1.65e-07","F=0.047, p=0.829"
iptm_ptm,"F=3.889, p=0.0492","F=3.720, p=0.0566","F=12.785, p=0.000433","F=2.362, p=0.126"
iptm,"F=3.837, p=0.0507","F=3.884, p=0.0515","F=12.281, p=0.000559","F=2.474, p=0.117"
pDockQ_mpDockQ,"F=26.957, p=3.02e-07","F=18.532, p=3.9e-05","F=18.582, p=2.5e-05","F=9.862, p=0.00192"
average_interface_pae,"F=3.045, p=0.0816","F=4.651, p=0.0334","F=12.295, p=0.000555","F=2.885, p=0.0908"
average_interface_plddt,"F=6.331, p=0.0122","F=11.601, p=0.00095","F=8.548, p=0.00384","F=7.518, p=0.00661"
binding_energy,"F=11.278, p=0.000844","F=3.035, p=0.0846","F=11.859, p=0.000692","F=6.423, p=0.012"
Num_intf_residues,"F=0.055, p=0.815","F=0.822, p=0.367","F=2.752, p=0.0987","F=0.200, p=0.656"
Polar,"F=5.150, p=0.0237","F=0.173, p=0.678","F=0.115, p=0.735","F=0.104, p=0.747"


In [17]:
# Save results to CSV in plots directory
output_dir = 'plots'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'anova_molecule_types.csv')
results_df_binding_comp_wcheck.to_csv(output_path)
print(f"ANOVA results saved to: {output_path}")

ANOVA results saved to: plots/anova_molecule_types.csv
